In [175]:
import shutil # Istrinam aplankus

shutil.rmtree('OpenImages')
shutil.rmtree('OpenImages_split')

In [176]:
!pip install openimages

In [177]:

import os
from openimages.download import download_dataset


data_dir = "OpenImages" # Nurodome direktorija
number_for_samples = 333
classes = ["Strawberry", "Bee", "Goldfish"] # Pasirinktos klases is OpenImages


if not os.path.exists(data_dir):
    os.makedirs(data_dir)


print("Downloading is starting...")
download_dataset(data_dir, classes, limit=number_for_samples) #Atsisiunciame nuotraukas

100%|██████████| 333/333 [00:19<00:00, 16.94it/s]


{'strawberry': {'images_dir': 'OpenImages/strawberry/images'},
 'bee': {'images_dir': 'OpenImages/bee/images'},
 'goldfish': {'images_dir': 'OpenImages/goldfish/images'}}

In [178]:
import random
import shutil

def split_dataset(base_dir, split_ratios=(0.7, 0.2, 0.1)):
    assert abs(sum(split_ratios) - 1.0) < 1e-6, "Ratios must sum to 1"
    sets = ['train', 'validate', 'test']

    for cls in os.listdir(base_dir):
        class_dir = os.path.join(base_dir, cls, "images")
        if not os.path.isdir(class_dir):
            continue

        images = [f for f in os.listdir(class_dir) if f.endswith('.jpg')]
        if not images:
            print(f"No images found for class: {cls}")
            continue

        random.shuffle(images)

        total = len(images)
        train_end = int(split_ratios[0] * total)
        val_end = train_end + int(split_ratios[1] * total)

        splits = {
            'train': images[:train_end],
            'validate': images[train_end:val_end],
            'test': images[val_end:]
        }

        for split_name in sets:
            split_dir = os.path.join(base_dir + "_split", split_name, cls)
            os.makedirs(split_dir, exist_ok=True)
            for img in splits[split_name]:
                src = os.path.join(class_dir, img)
                dst = os.path.join(split_dir, img)
                shutil.copy(src, dst)

    print("Dataset successfully split into train, validate, and test!")

split_dataset(data_dir)

Dataset successfully split into train, validate, and test!


In [179]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

In [180]:
class Lab2Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

In [181]:
transform = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),  # Resize first → faster aug
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),  # Smaller angle
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]),
    'validation': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]),
    'test': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
}

In [182]:
train_dataset = Lab2Dataset(data_dir='OpenImages_split/train', transform = transform['train'])
val_dataset = Lab2Dataset(data_dir='OpenImages_split/validate', transform = transform['validation'])
test_dataset = Lab2Dataset(data_dir='OpenImages_split/test', transform = transform['test'])

In [183]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [184]:
import torch.nn.functional as F

class Lab2CNN(nn.Module):
    def __init__(self, num_classes=3):
        super(Lab2CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)
        x = self.dropout1(F.leaky_relu(self.fc1(x)))
        x = self.dropout2(F.leaky_relu(self.fc2(x)))
        x = self.fc3(x)

        return x


In [185]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Lab2CNN(num_classes=3)
model.to(device)

Lab2CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=3, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [186]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [187]:
num_epoch = 30
train_losses, val_losses = [], []

In [188]:
for epoch in range(num_epoch):
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_dataloader.dataset)
    train_losses.append(train_loss)


    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
    val_loss = running_loss / len(val_dataloader.dataset)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epoch} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/30 - Train Loss: 3.5999 - Val Loss: 0.9511
Epoch 2/30 - Train Loss: 1.0915 - Val Loss: 0.8957
Epoch 3/30 - Train Loss: 0.8811 - Val Loss: 0.7865
Epoch 4/30 - Train Loss: 0.8226 - Val Loss: 0.7415
Epoch 5/30 - Train Loss: 0.7104 - Val Loss: 0.6862
Epoch 6/30 - Train Loss: 0.7290 - Val Loss: 0.6642
Epoch 7/30 - Train Loss: 0.7136 - Val Loss: 0.6160
Epoch 8/30 - Train Loss: 0.6963 - Val Loss: 0.6689
Epoch 9/30 - Train Loss: 0.6182 - Val Loss: 0.6829
Epoch 10/30 - Train Loss: 0.5936 - Val Loss: 0.6676
Epoch 11/30 - Train Loss: 0.6733 - Val Loss: 0.5462
Epoch 12/30 - Train Loss: 0.5790 - Val Loss: 0.5098
Epoch 13/30 - Train Loss: 0.5421 - Val Loss: 0.4899
Epoch 14/30 - Train Loss: 0.5523 - Val Loss: 0.4609
Epoch 15/30 - Train Loss: 0.4767 - Val Loss: 0.6216
Epoch 16/30 - Train Loss: 0.5051 - Val Loss: 0.4825
Epoch 17/30 - Train Loss: 0.4635 - Val Loss: 0.6148
Epoch 18/30 - Train Loss: 0.3883 - Val Loss: 0.8437
Epoch 19/30 - Train Loss: 0.4260 - Val Loss: 0.5252
Epoch 20/30 - Train L

In [189]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np


model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())


class_labels = test_dataset.classes

conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
precisions = precision_score(y_true, y_pred, average=None)
recalls = recall_score(y_true, y_pred, average=None)
f1s = f1_score(y_true, y_pred, average=None)


print("Confusion Matrix:")
print(conf_matrix)
print(f"\nTikslumas (Accuracy): {accuracy:.4f}\n")

print("Rezultatai pagal klases:")
for i, label in enumerate(class_labels):
    print(f"{label}")
    print(f"  Precizija (Precision): {precisions[i]:.4f}")
    print(f"  Atkūrimas (Recall):    {recalls[i]:.4f}")
    print(f"  F1 reikšmė:            {f1s[i]:.4f}\n")



Confusion Matrix:
[[27  5  2]
 [ 1 33  0]
 [ 0  6 28]]

Tikslumas (Accuracy): 0.8627

Rezultatai pagal klases:
bee
  Precizija (Precision): 0.9643
  Atkūrimas (Recall):    0.7941
  F1 reikšmė:            0.8710

goldfish
  Precizija (Precision): 0.7500
  Atkūrimas (Recall):    0.9706
  F1 reikšmė:            0.8462

strawberry
  Precizija (Precision): 0.9333
  Atkūrimas (Recall):    0.8235
  F1 reikšmė:            0.8750

